# SegNet Training Runner

This notebook is designed to run `train_SegNet.py` for training a segmentation network. It includes steps for setting parameters, preparing the dataset, running the training script, and monitoring the output.

## 1. Import Required Libraries
Import necessary Python libraries such as os, subprocess, and any dependencies required by `train_SegNet.py`.

In [1]:
# Import Required Libraries
import os
import sys
import subprocess
import json
import matplotlib.pyplot as plt


## 2. Set Training Parameters
Define and configure training parameters such as batch size, learning rate, number of epochs, and data paths.

In [67]:
# Set training parameters
batch_size = 8
learning_rate = 0.001
epochs = 50
data_dir = '../input/coffee-images/data'  # Adjust as needed
output_dir = './output/'
os.makedirs(output_dir, exist_ok=True)

# You can add more parameters as needed

In [88]:
!rm coffee_images/ -r

In [89]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
token = user_secrets.get_secret("GITHUB_KEY")

!git clone https://{token}@github.com/PlayerPlanet/coffee_images.git

Cloning into 'coffee_images'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 70 (delta 35), reused 57 (delta 22), pack-reused 0 (from 0)
Receiving objects: 100% (70/70), 2.76 MiB | 33.30 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [90]:
!pip install /kaggle/working/coffee_images

Processing ./coffee_images
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for coffee-images: filename=coffee_images-0.1.0-py3-none-any.whl size=2807672 sha256=3e15d146579b0ea78964207d83321f3f37bdf6636770aa2da6b84347ce5030db
  Stored in directory: /tmp/pip-ephem-wheel-cache-he75skyx/wheels/80/c2/4c/3863249e107ad71db92bfbbc9ba4f35cf8fd63937ff98415fe
Successfully built coffee-images
  Attempting uninstall: coffee-images
    Found existing installation: coffee-images 0.1.0
    Uninstalling coffee-images-0.1.0:
      Successfully uninstalled coffee-images-0.1.0


## 3. Prepare Dataset
Ensure the dataset is available and properly structured for SegNet training. Optionally, include code to check or preprocess the dataset.

In [70]:
# Check if dataset directories exist
assert os.path.exists(data_dir+"/features/"), f"Feture directory not found: {data_dir}features/"
assert os.path.exists(data_dir+"/labels/"), f"Mask directory not found: {data_dir}labels/"

# Optionally, add preprocessing or dataset checks here

In [86]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.6.0+cu124
True


## 4. Run train_SegNet.py Script
Use Python's subprocess module or the %run magic command to execute `train_SegNet.py` with the specified parameters.

In [91]:
# Run train_SegNet.py with parameters
script_path = os.path.join(os.path.dirname(os.getcwd()), 'working/coffee_images/coffee_images/training', 'train_SegNet.py')
cmd = [sys.executable, script_path,
       '--batch_size', str(batch_size),
       '--learning_rate', str(learning_rate),
       '--epochs', str(epochs),
       '--data_dir', data_dir]

print('Running command:', ' '.join(cmd))
process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
output_lines = []
for line in process.stdout:
    print(line, end='')
    output_lines.append(line)
process.wait()

# Save output log
with open(os.path.join(output_dir, 'train_log.txt'), 'w') as f:
    f.writelines(output_lines)


Running command: /usr/bin/python3 /kaggle/working/coffee_images/coffee_images/training/train_SegNet.py --batch_size 8 --learning_rate 0.001 --epochs 50 --data_dir ../input/coffee-images/data
Creating new checkpoint '/kaggle/working/segnet_weights.pth.tar'
Epoch 1:

Epoch 1/50:   0%|          | 0/30 [00:00<?, ?it/s]Loss at 1 mini-batch: 0.4328208863735199

Epoch 1/50:   3%|▎         | 1/30 [00:02<00:59,  2.04s/it]Loss at 2 mini-batch: 0.43031391501426697

Epoch 1/50:   7%|▋         | 2/30 [00:02<00:38,  1.38s/it]Loss at 3 mini-batch: 0.426779568195343

Epoch 1/50:  10%|█         | 3/30 [00:03<00:31,  1.16s/it]Loss at 4 mini-batch: 0.41958877444267273

Epoch 1/50:  13%|█▎        | 4/30 [00:04<00:27,  1.06s/it]Loss at 5 mini-batch: 0.4095768332481384

Epoch 1/50:  17%|█▋        | 5/30 [00:05<00:25,  1.01s/it]Loss at 6 mini-batch: 0.4011789560317993

Epoch 1/50:  20%|██        | 6/30 [00:06<00:23,  1.03it/s]Loss at 7 mini-batch: 0.3872644305229187

Epoch 1/50:  23%|██▎       | 7/30 [00:07<

## 5. Monitor Training Output
Capture and display the output logs from the training script, and optionally visualize training metrics such as loss and accuracy.

In [ ]:
# Visualize training metrics if available
import re
log_file = os.path.join(output_dir, 'train_log.txt')
losses = []
epochs_list = []
if os.path.exists(log_file):
    with open(log_file, 'r') as f:
        for line in f:
            # Example: "Epoch 1/50 - loss: 0.345"
            match = re.search(r"Loss at ([0-9]+) mini-batch: ([0-9.]+)", line)
            if match:
                epochs_list.append(int(match.group(1)))
                losses.append(float(match.group(2)))
if losses:
    plt.figure(figsize=(8,4))
    plt.plot(losses, marker='o')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss Curve')
    plt.grid(True)
    plt.show()
else:
    print('No loss data found in log.')

: 